In [77]:
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.33)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

In [78]:
# keras 或 tensorflow 调用GPU报错：Blas GEMM launch failed
# https://blog.csdn.net/Leo_Xu06/article/details/82023330
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 

In [79]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

C:\Users\lee\AppData\Local\conda\conda\envs\deep\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\lee\AppData\Local\conda\conda\envs\deep\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [80]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [81]:
# Part 2 - Now let's make the ANN!
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_shape=(11, )))
model.add(layers.Dense(6, kernel_initializer = 'uniform', activation = 'relu'))
model.add(layers.Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# model_hist= model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_dev, y_dev))
model_hist= model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_test, y_test))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 0s 57us/step - loss: 0.6720 - acc: 0.7926 - val_loss: 0.6448 - val_acc: 0.7975
Epoch 2/10
8000/8000 [==============================] - 0s 23us/step - loss: 0.6036 - acc: 0.7960 - val_loss: 0.5585 - val_acc: 0.7975
Epoch 3/10
8000/8000 [==============================] - 0s 23us/step - loss: 0.5152 - acc: 0.7960 - val_loss: 0.4805 - val_acc: 0.7975
Epoch 4/10
8000/8000 [==============================] - 0s 24us/step - loss: 0.4596 - acc: 0.7960 - val_loss: 0.4479 - val_acc: 0.7975
Epoch 5/10
8000/8000 [==============================] - 0s 24us/step - loss: 0.4410 - acc: 0.7960 - val_loss: 0.4342 - val_acc: 0.7975
Epoch 6/10
8000/8000 [==============================] - 0s 23us/step - loss: 0.4317 - acc: 0.7960 - val_loss: 0.4267 - val_acc: 0.7975
Epoch 7/10
8000/8000 [==============================] - 0s 23us/step - loss: 0.4265 - acc: 0.7960 - val_loss: 0.4228 - val_acc: 0.7975
Epoch 8

In [82]:
y_pred = model.predict(X_test)
y_pred = y_pred > 0.5
y_pred = y_pred.astype(int).ravel()
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1545,   50],
       [ 298,  107]], dtype=int64)

In [83]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    model = models.Sequential()
    model.add(layers.Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_shape=(11, )))
    model.add(layers.Dense(6, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model
model = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

Epoch 1/100
5333/5333 [==============================] - 1s 131us/step - loss: 0.6209 - acc: 0.7949
Epoch 2/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.4636 - acc: 0.7980
Epoch 3/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4368 - acc: 0.7980
Epoch 4/100
5333/5333 [==============================] - 0s 87us/step - loss: 0.4310 - acc: 0.7980
Epoch 5/100
5333/5333 [==============================] - 0s 86us/step - loss: 0.4277 - acc: 0.7980
Epoch 6/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4254 - acc: 0.7980
Epoch 7/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.4233 - acc: 0.7980
Epoch 8/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4216 - acc: 0.7980
Epoch 9/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.4200 - acc: 0.7980
Epoch 10/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.4186 - acc: 0.8106
Epoch 11

5333/5333 [==============================] - 0s 84us/step - loss: 0.3982 - acc: 0.8350
Epoch 84/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.3978 - acc: 0.8372
Epoch 85/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3979 - acc: 0.8352
Epoch 86/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3973 - acc: 0.8372
Epoch 87/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3972 - acc: 0.8361
Epoch 88/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.3972 - acc: 0.8359
Epoch 89/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3972 - acc: 0.8354
Epoch 90/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3967 - acc: 0.8361
Epoch 91/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3965 - acc: 0.8372
Epoch 92/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.3962 - acc: 0.8354
Epoch 93/100


5333/5333 [==============================] - 0s 84us/step - loss: 0.4031 - acc: 0.8329
Epoch 66/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4028 - acc: 0.8312
Epoch 67/100
5333/5333 [==============================] - 0s 82us/step - loss: 0.4030 - acc: 0.8318
Epoch 68/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4030 - acc: 0.8311
Epoch 69/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.4029 - acc: 0.8316
Epoch 70/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.4030 - acc: 0.8326
Epoch 71/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.4023 - acc: 0.8320
Epoch 72/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4025 - acc: 0.8335
Epoch 73/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4026 - acc: 0.8337
Epoch 74/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.4025 - acc: 0.8331
Epoch 75/100


5334/5334 [==============================] - 0s 85us/step - loss: 0.4018 - acc: 0.8358
Epoch 48/100
5334/5334 [==============================] - 0s 85us/step - loss: 0.4019 - acc: 0.8367
Epoch 49/100
5334/5334 [==============================] - 0s 83us/step - loss: 0.4016 - acc: 0.8369
Epoch 50/100
5334/5334 [==============================] - 0s 83us/step - loss: 0.4012 - acc: 0.8365
Epoch 51/100
5334/5334 [==============================] - 0s 84us/step - loss: 0.4014 - acc: 0.8380
Epoch 52/100
5334/5334 [==============================] - 0s 84us/step - loss: 0.4010 - acc: 0.8376
Epoch 53/100
5334/5334 [==============================] - 0s 84us/step - loss: 0.4013 - acc: 0.8365
Epoch 54/100
5334/5334 [==============================] - 0s 82us/step - loss: 0.4011 - acc: 0.8367
Epoch 55/100
5334/5334 [==============================] - 0s 85us/step - loss: 0.4011 - acc: 0.8376
Epoch 56/100
5334/5334 [==============================] - 0s 84us/step - loss: 0.4008 - acc: 0.8369
Epoch 57/100


5333/5333 [==============================] - 0s 85us/step - loss: 0.3670 - acc: 0.8491
Epoch 30/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.3648 - acc: 0.8504
Epoch 31/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3637 - acc: 0.8509
Epoch 32/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.3619 - acc: 0.8519
Epoch 33/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.3601 - acc: 0.8524
Epoch 34/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.3592 - acc: 0.8521
Epoch 35/100
5333/5333 [==============================] - 0s 84us/step - loss: 0.3580 - acc: 0.8537
Epoch 36/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.3568 - acc: 0.8549
Epoch 37/100
5333/5333 [==============================] - 0s 83us/step - loss: 0.3563 - acc: 0.8528
Epoch 38/100
5333/5333 [==============================] - 0s 85us/step - loss: 0.3554 - acc: 0.8560
Epoch 39/100


...

Epoch 111/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3926 - acc: 0.8391
Epoch 112/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.3926 - acc: 0.8385
Epoch 113/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3926 - acc: 0.8394
Epoch 114/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3928 - acc: 0.8381
Epoch 115/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.3928 - acc: 0.8390
Epoch 116/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3925 - acc: 0.8379
Epoch 117/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3928 - acc: 0.8387
Epoch 118/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.3924 - acc: 0.8379
Epoch 119/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3926 - acc: 0.8395
Epoch 120/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3920 - ac

8000/8000 [==============================] - 1s 86us/step - loss: 0.3921 - acc: 0.8375
Epoch 193/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.3918 - acc: 0.8380
Epoch 194/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3919 - acc: 0.8382
Epoch 195/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3921 - acc: 0.8371
Epoch 196/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3921 - acc: 0.8402
Epoch 197/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3921 - acc: 0.8389
Epoch 198/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.3921 - acc: 0.8394
Epoch 199/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3920 - acc: 0.8395
Epoch 200/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3921 - acc: 0.8387
Epoch 201/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3923 - acc: 0.8387
Epoc

8000/8000 [==============================] - 1s 84us/step - loss: 0.3891 - acc: 0.8406
Epoch 274/500
8000/8000 [==============================] - 1s 87us/step - loss: 0.3886 - acc: 0.8401
Epoch 275/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3890 - acc: 0.8406
Epoch 276/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3888 - acc: 0.8411
Epoch 277/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3884 - acc: 0.8405
Epoch 278/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3879 - acc: 0.8422
Epoch 279/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3882 - acc: 0.8407
Epoch 280/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.3876 - acc: 0.8400
Epoch 281/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3882 - acc: 0.8406
Epoch 282/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3878 - acc: 0.8422
Epoc

8000/8000 [==============================] - 1s 86us/step - loss: 0.3370 - acc: 0.8647
Epoch 355/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3376 - acc: 0.8651
Epoch 356/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3365 - acc: 0.8646
Epoch 357/500
8000/8000 [==============================] - 1s 84us/step - loss: 0.3366 - acc: 0.8657
Epoch 358/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.3369 - acc: 0.8664
Epoch 359/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3366 - acc: 0.8646
Epoch 360/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3367 - acc: 0.8652
Epoch 361/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3369 - acc: 0.8654
Epoch 362/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.3365 - acc: 0.8661
Epoch 363/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.3365 - acc: 0.8661
Epoc

8000/8000 [==============================] - 1s 89us/step - loss: 0.3340 - acc: 0.8651
Epoch 436/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.3340 - acc: 0.8639
Epoch 437/500
8000/8000 [==============================] - 1s 87us/step - loss: 0.3345 - acc: 0.8650
Epoch 438/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.3347 - acc: 0.8664
Epoch 439/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.3343 - acc: 0.8644
Epoch 440/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.3339 - acc: 0.8646
Epoch 441/500
8000/8000 [==============================] - 1s 90us/step - loss: 0.3340 - acc: 0.8651
Epoch 442/500
8000/8000 [==============================] - 1s 90us/step - loss: 0.3342 - acc: 0.8642
Epoch 443/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.3341 - acc: 0.8657
Epoch 444/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3338 - acc: 0.8647
Epoc

In [87]:
from sklearn.metrics import accuracy_score
final = grid_search.best_estimator_
print(accuracy_score(y_train, final.predict(X_train)))
print(accuracy_score(y_test, final.predict(X_test)))

0.86675
0.8625


In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    model = models.Sequential()
    model.add(layers.Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_shape=(11, )))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model
model = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

Epoch 1/100
5333/5333 [==============================] - 1s 217us/step - loss: 0.6230 - acc: 0.7971
Epoch 2/100
5333/5333 [==============================] - 1s 103us/step - loss: 0.4959 - acc: 0.7980
Epoch 3/100
5333/5333 [==============================] - 1s 102us/step - loss: 0.4685 - acc: 0.7980
Epoch 4/100
5333/5333 [==============================] - 1s 102us/step - loss: 0.4680 - acc: 0.7980
Epoch 5/100
5333/5333 [==============================] - 1s 100us/step - loss: 0.4653 - acc: 0.7980
Epoch 6/100
5333/5333 [==============================] - 1s 102us/step - loss: 0.4644 - acc: 0.7980
Epoch 7/100
5333/5333 [==============================] - 1s 99us/step - loss: 0.4676 - acc: 0.7980
Epoch 8/100
5333/5333 [==============================] - 1s 100us/step - loss: 0.4669 - acc: 0.7980
Epoch 9/100
5333/5333 [==============================] - 1s 99us/step - loss: 0.4668 - acc: 0.7980
Epoch 10/100
5333/5333 [==============================] - 1s 99us/step - loss: 0.4583 - acc: 0.7980
Ep

5333/5333 [==============================] - 1s 94us/step - loss: 0.4633 - acc: 0.7980
Epoch 83/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4588 - acc: 0.7980
Epoch 84/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4642 - acc: 0.7980
Epoch 85/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4607 - acc: 0.7980
Epoch 86/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4634 - acc: 0.7980
Epoch 87/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4591 - acc: 0.7980
Epoch 88/100
5333/5333 [==============================] - 1s 95us/step - loss: 0.4621 - acc: 0.7980
Epoch 89/100
5333/5333 [==============================] - 0s 93us/step - loss: 0.4622 - acc: 0.7980
Epoch 90/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4576 - acc: 0.7980
Epoch 91/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4598 - acc: 0.7980
Epoch 92/100


5333/5333 [==============================] - 1s 94us/step - loss: 0.4642 - acc: 0.7935
Epoch 64/100
5333/5333 [==============================] - 1s 96us/step - loss: 0.4618 - acc: 0.7935
Epoch 65/100
5333/5333 [==============================] - 0s 93us/step - loss: 0.4707 - acc: 0.7935
Epoch 66/100
5333/5333 [==============================] - 0s 94us/step - loss: 0.4629 - acc: 0.7935
Epoch 67/100
5333/5333 [==============================] - 1s 96us/step - loss: 0.4651 - acc: 0.7935
Epoch 68/100
5333/5333 [==============================] - 1s 95us/step - loss: 0.4639 - acc: 0.7935
Epoch 69/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4657 - acc: 0.7935
Epoch 70/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4596 - acc: 0.7935
Epoch 71/100
5333/5333 [==============================] - 1s 95us/step - loss: 0.4695 - acc: 0.7935
Epoch 72/100
5333/5333 [==============================] - 0s 93us/step - loss: 0.4623 - acc: 0.7935
Epoch 73/100


5334/5334 [==============================] - 1s 94us/step - loss: 0.4587 - acc: 0.7964
Epoch 46/100
5334/5334 [==============================] - 1s 94us/step - loss: 0.4620 - acc: 0.7964
Epoch 47/100
5334/5334 [==============================] - 1s 97us/step - loss: 0.4695 - acc: 0.7964
Epoch 48/100
5334/5334 [==============================] - 1s 95us/step - loss: 0.4623 - acc: 0.7964
Epoch 49/100
5334/5334 [==============================] - 1s 96us/step - loss: 0.4631 - acc: 0.7964
Epoch 50/100
5334/5334 [==============================] - 1s 96us/step - loss: 0.4600 - acc: 0.7964
Epoch 51/100
5334/5334 [==============================] - 1s 96us/step - loss: 0.4553 - acc: 0.7964
Epoch 52/100
5334/5334 [==============================] - 1s 94us/step - loss: 0.4689 - acc: 0.7964
Epoch 53/100
5334/5334 [==============================] - 1s 98us/step - loss: 0.4633 - acc: 0.7964
Epoch 54/100
5334/5334 [==============================] - 0s 93us/step - loss: 0.4656 - acc: 0.7964
Epoch 55/100


5333/5333 [==============================] - 0s 93us/step - loss: 0.4702 - acc: 0.7980
Epoch 28/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4571 - acc: 0.7980
Epoch 29/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4635 - acc: 0.7980
Epoch 30/100
5333/5333 [==============================] - 1s 97us/step - loss: 0.4567 - acc: 0.7980
Epoch 31/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4606 - acc: 0.7980
Epoch 32/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4616 - acc: 0.7980
Epoch 33/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4592 - acc: 0.7980
Epoch 34/100
5333/5333 [==============================] - 1s 94us/step - loss: 0.4620 - acc: 0.7980
Epoch 35/100
5333/5333 [==============================] - 1s 97us/step - loss: 0.4616 - acc: 0.7980
Epoch 36/100
5333/5333 [==============================] - 1s 96us/step - loss: 0.4555 - acc: 0.7980
Epoch 37/100


...

5334/5334 [==============================] - 1s 97us/step - loss: 0.4582 - acc: 0.7964
Epoch 266/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4577 - acc: 0.7964
Epoch 267/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4568 - acc: 0.7964
Epoch 268/500
5334/5334 [==============================] - 1s 95us/step - loss: 0.4568 - acc: 0.7964
Epoch 269/500
5334/5334 [==============================] - 1s 94us/step - loss: 0.4595 - acc: 0.7964
Epoch 270/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4521 - acc: 0.7964
Epoch 271/500
5334/5334 [==============================] - 1s 94us/step - loss: 0.4566 - acc: 0.7964
Epoch 272/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4622 - acc: 0.7964
Epoch 273/500
5334/5334 [==============================] - 1s 94us/step - loss: 0.4575 - acc: 0.7964
Epoch 274/500
5334/5334 [==============================] - 1s 94us/step - loss: 0.4597 - acc: 0.7964
Epoc

5334/5334 [==============================] - 1s 97us/step - loss: 0.4613 - acc: 0.7964
Epoch 347/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4562 - acc: 0.7964
Epoch 348/500
5334/5334 [==============================] - 1s 94us/step - loss: 0.4594 - acc: 0.7964
Epoch 349/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4590 - acc: 0.7964
Epoch 350/500
5334/5334 [==============================] - 1s 94us/step - loss: 0.4566 - acc: 0.7964
Epoch 351/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4598 - acc: 0.7964
Epoch 352/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4569 - acc: 0.7964
Epoch 353/500
5334/5334 [==============================] - 1s 95us/step - loss: 0.4607 - acc: 0.7964
Epoch 354/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4564 - acc: 0.7964
Epoch 355/500
5334/5334 [==============================] - 1s 96us/step - loss: 0.4599 - acc: 0.7964
Epoc

5334/5334 [==============================] - 1s 94us/step - loss: 0.4545 - acc: 0.7964
Epoch 428/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4603 - acc: 0.7964
Epoch 429/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4574 - acc: 0.7964
Epoch 430/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4579 - acc: 0.7964
Epoch 431/500
5334/5334 [==============================] - 1s 95us/step - loss: 0.4576 - acc: 0.7964
Epoch 432/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4531 - acc: 0.7964
Epoch 433/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4552 - acc: 0.7964
Epoch 434/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4626 - acc: 0.7964
Epoch 435/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4574 - acc: 0.7964
Epoch 436/500
5334/5334 [==============================] - 1s 97us/step - loss: 0.4612 - acc: 0.7964
Epoc

5333/5333 [==============================] - 1s 97us/step - loss: 0.4592 - acc: 0.7980
Epoch 9/500
5333/5333 [==============================] - 1s 98us/step - loss: 0.4575 - acc: 0.7980
Epoch 10/500
5333/5333 [==============================] - 1s 94us/step - loss: 0.4623 - acc: 0.7980
Epoch 11/500
5333/5333 [==============================] - 1s 97us/step - loss: 0.4604 - acc: 0.7980
Epoch 12/500
5333/5333 [==============================] - 1s 98us/step - loss: 0.4614 - acc: 0.7980
Epoch 13/500
5333/5333 [==============================] - 1s 96us/step - loss: 0.4600 - acc: 0.7980
Epoch 14/500
5333/5333 [==============================] - 1s 97us/step - loss: 0.4569 - acc: 0.7980
Epoch 15/500
5333/5333 [==============================] - 1s 94us/step - loss: 0.4566 - acc: 0.7980
Epoch 16/500
5333/5333 [==============================] - 1s 97us/step - loss: 0.4590 - acc: 0.7980
Epoch 17/500
5333/5333 [==============================] - 1s 98us/step - loss: 0.4538 - acc: 0.7980
Epoch 18/500
5

5333/5333 [==============================] - 1s 101us/step - loss: 0.4554 - acc: 0.7980
Epoch 90/500
5333/5333 [==============================] - 1s 102us/step - loss: 0.4576 - acc: 0.7980
Epoch 91/500
5333/5333 [==============================] - 1s 99us/step - loss: 0.4540 - acc: 0.7980
Epoch 92/500
5333/5333 [==============================] - 1s 102us/step - loss: 0.4544 - acc: 0.7980
Epoch 93/500
5333/5333 [==============================] - 1s 101us/step - loss: 0.4639 - acc: 0.7980
Epoch 94/500
5333/5333 [==============================] - 1s 106us/step - loss: 0.4569 - acc: 0.7980
Epoch 95/500
5333/5333 [==============================] - 1s 107us/step - loss: 0.4561 - acc: 0.7980
Epoch 96/500
5333/5333 [==============================] - 1s 102us/step - loss: 0.4588 - acc: 0.7980
Epoch 97/500
5333/5333 [==============================] - 1s 101us/step - loss: 0.4587 - acc: 0.7980
Epoch 98/500
5333/5333 [==============================] - 1s 103us/step - loss: 0.4531 - acc: 0.7980
Epoc

In [ ]:
from sklearn.metrics import accuracy_score
final = grid_search.best_estimator_
print(accuracy_score(y_train, final.predict(X_train)))
print(accuracy_score(y_test, final.predict(X_test)))

In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
def build_classifier(optimizer):
    model = models.Sequential()
    model.add(layers.Dense(6, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu', input_shape=(11, )))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model
model = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

# from keras import regularizers
# model.add(Dense(64, input_dim=64,
#                 kernel_regularizer=regularizers.l2(0.01),
#                 activity_regularizer=regularizers.l1(0.01)))
# keras.regularizers.l1(0.)
# keras.regularizers.l2(0.)
# keras.regularizers.l1_l2(l1=0.01, l2=0.01)

In [ ]:
from sklearn.metrics import accuracy_score
final = grid_search.best_estimator_
print(accuracy_score(y_train, final.predict(X_train)))
print(accuracy_score(y_test, final.predict(X_test)))

In [ ]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
def build_classifier(optimizer):
    model = models.Sequential()
    model.add(layers.Dense(24, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu', input_shape=(11, )))
    model.add(layers.Dense(24, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dense(24, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dense(24, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dense(24, kernel_regularizer=regularizers.l2(0.01) ,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(layers.Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model
model = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters)
print(best_accuracy)

# from keras import regularizers
# model.add(Dense(64, input_dim=64,
#                 kernel_regularizer=regularizers.l2(0.01),
#                 activity_regularizer=regularizers.l1(0.01)))
# keras.regularizers.l1(0.)
# keras.regularizers.l2(0.)
# keras.regularizers.l1_l2(l1=0.01, l2=0.01)

In [ ]:
from sklearn.metrics import accuracy_score
final = grid_search.best_estimator_
print(accuracy_score(y_train, final.predict(X_train)))
print(accuracy_score(y_test, final.predict(X_test)))